# Modeling connections between neurons

Next, let's introduce modeling synaptic connections and making our neuron models affect each other.

First, load brian2 as usual

In [ ]:
from brian2 import *
%matplotlib inline

### Next, let's define our normal equations and constants
For now, we'll keep all of our variables identical to our single-neuron simulation.

In [ ]:
neuron_equation = '''
dv/dt = (1/Cap_mem) * (((v_0 - v)/Resist_mem) + I) : volt (unless refractory)
I: amp (constant)
'''

Cap_mem = 0.002*uF
v_0 = -70*mV 
Resist_mem = 10*Mohm


### Finally, let's set up a simple two-neuron simulation. 

We won't connect them yet, for now let's just set up more than one neuron group.

Below is a copy of the simulation we did with a single neuron, but with each component duplicated to create two groups of one neuron each. Each of these need to be monitored and plotted separately.

The only additional

In [ ]:
start_scope()

neuron_number = 1
threshold_voltage = 'v > -55*mV'
reset_v = 'v = v_0'
refractory_period = 2*ms
solution_method = 'heun'
sim_neuron_1 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_1.v = v_0
sim_neuron_1.I = 2*namp

sim_neuron_2 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_2.v = v_0
sim_neuron_2.I = 2*namp


voltage_mon_1 = StateMonitor(sim_neuron_1, 'v', record=True)
spike_mon_1 = SpikeMonitor(sim_neuron_1, record=True)

voltage_mon_2 = StateMonitor(sim_neuron_2, 'v', record=True)
spike_mon_2 = SpikeMonitor(sim_neuron_2, record=True)

run(100*ms)

plot(voltage_mon_1.t/ms, voltage_mon_1.v[0], c='b')
for t in spike_mon_1.t:
    axvline(t/ms, c='b', lw=1)

plot(voltage_mon_2.t/ms, voltage_mon_2.v[0], c='y')
for t in spike_mon_2.t:
    axvline(t/ms, c='y', lw=1)

ylim(-0.1, 0.04)
xlabel('Time (ms)')
ylabel('v')

### You'll notice that we can only see one output (shown in yellow here). 
That's because the outputs of these two neurons are identical. If we want to plot them separately, we have to show() each graph.

In [ ]:
start_scope()

neuron_number = 1
threshold_voltage = 'v > -55*mV'
reset_v = 'v = v_0'
refractory_period = 2*ms
solution_method = 'heun'
sim_neuron_1 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_1.v = v_0
sim_neuron_1.I = 2*namp

sim_neuron_2 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_2.v = v_0
sim_neuron_2.I = 2*namp

voltage_mon_1 = StateMonitor(sim_neuron_1, 'v', record=True)
spike_mon_1 = SpikeMonitor(sim_neuron_1, record=True)

voltage_mon_2 = StateMonitor(sim_neuron_2, 'v', record=True)
spike_mon_2 = SpikeMonitor(sim_neuron_2, record=True)

run(100*ms)

plot(voltage_mon_1.t/ms, voltage_mon_1.v[0], c='b')
for t in spike_mon_1.t:
    axvline(t/ms, c='b', lw=1)
ylim(-0.1, 0.04)
xlabel('Time (ms)')
ylabel('v')

show()

plot(voltage_mon_2.t/ms, voltage_mon_2.v[0], c='y')
for t in spike_mon_1.t:
    axvline(t/ms, c='y', lw=1)

ylim(-0.1, 0.04)
xlabel('Time (ms)')
ylabel('v')
show()

### So if you want to track multiple neurons graphically, it may be necessary to plot them separately.

Alternatively, you can make the second of the plotted neurons a dashed line. We'll do that here, then stick with that so that we can see the behavior of the two neurons side by side as we connect them

In [ ]:
start_scope()

neuron_number = 1
threshold_voltage = 'v > -55*mV'
reset_v = 'v = v_0'
refractory_period = 2*ms
solution_method = 'heun'
sim_neuron_1 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_1.v = v_0
sim_neuron_1.I = 2*namp

sim_neuron_2 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_2.v = v_0
sim_neuron_2.I = 2*namp

voltage_mon_1 = StateMonitor(sim_neuron_1, 'v', record=True)
spike_mon_1 = SpikeMonitor(sim_neuron_1, record=True)

voltage_mon_2 = StateMonitor(sim_neuron_2, 'v', record=True)
spike_mon_2 = SpikeMonitor(sim_neuron_2, record=True)

run(100*ms)

plot(voltage_mon_1.t/ms, voltage_mon_1.v[0], c='b')
for t in spike_mon_1.t:
    axvline(t/ms, c='b', lw=2)

plot(voltage_mon_2.t/ms, voltage_mon_2.v[0], c='y', ls='--')
for t in spike_mon_1.t:
    axvline(t/ms, c='y', lw=2, ls='--')

ylim(-0.1, 0.04)
xlabel('Time (ms)')
ylabel('v')
show()

### Now that we can see both neuron traces on the same plot, let's connect them and watch them interact

To do that, we have have to add a new "Synapses" group and connect the two neurons in order. I'll connect the only neuron in group 1 to the only neuron in group 2.

We also need to define how the synapse in that group will function. To keep things simple, we'll define synapses by the voltage change the presynaptic cell causes in the postsynaptic.

In [ ]:
start_scope()

neuron_number = 1
threshold_voltage = 'v > -55*mV'
reset_v = 'v = v_0'
refractory_period = 2*ms
solution_method = 'heun'
sim_neuron_1 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_1.v = v_0
sim_neuron_1.I = 2*namp

sim_neuron_2 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_2.v = v_0
sim_neuron_2.I = 2*namp


# NEW CODE BEGINS HERE

excite_volt = '3*mV'

excite_synapse = Synapses(sim_neuron_1, sim_neuron_2, 'weight: volt', on_pre='v_post+=weight')
excite_synapse.connect(i=[0], j=[0])
excite_synapse.weight = excite_volt

#NEW CODE ENDS HERE



voltage_mon_1 = StateMonitor(sim_neuron_1, 'v', record=True)
spike_mon_1 = SpikeMonitor(sim_neuron_1, record=True)

voltage_mon_2 = StateMonitor(sim_neuron_2, 'v', record=True)
spike_mon_2 = SpikeMonitor(sim_neuron_2, record=True)

run(100*ms)

plot(voltage_mon_1.t/ms, voltage_mon_1.v[0], c='b')
for t in spike_mon_1.t:
    axvline(t/ms, c='b', lw=2)

plot(voltage_mon_2.t/ms, voltage_mon_2.v[0], c='y', ls='--')
for t in spike_mon_1.t:
    axvline(t/ms, c='y', lw=2, ls='--')

ylim(-0.1, 0.04)
xlabel('Time (ms)')
ylabel('v')
show()

### Surprise!

Don't forget: we can only affect neurons when they **aren't** in their refractory period. If neurons 1 and 2 already fire at the same time, it won't matter if neuron 1 excites neuron 2. It already fired and is reset at its refractory period.

Let's make the current input to neuron 2 zero so that the only input it receives is from neuron 1.

In [ ]:
start_scope()

neuron_number = 1
threshold_voltage = 'v > -55*mV'
reset_v = 'v = v_0'
refractory_period = 2*ms
solution_method = 'heun'
sim_neuron_1 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_1.v = v_0
sim_neuron_1.I = 2*namp

sim_neuron_2 = NeuronGroup(neuron_number, neuron_equation, threshold = threshold_voltage, 
                         reset = reset_v, refractory = refractory_period, method = 'heun')
sim_neuron_2.v = v_0
sim_neuron_2.I = 0*namp

excite_volt = '3*mV'

excite_synapse = Synapses(sim_neuron_1, sim_neuron_2, 'weight: volt', on_pre='v_post+=weight')
excite_synapse.connect(i=[0], j=[0])
excite_synapse.weight = excite_volt


voltage_mon_1 = StateMonitor(sim_neuron_1, 'v', record=True)
spike_mon_1 = SpikeMonitor(sim_neuron_1, record=True)

voltage_mon_2 = StateMonitor(sim_neuron_2, 'v', record=True)
spike_mon_2 = SpikeMonitor(sim_neuron_2, record=True)

run(100*ms)

plot(voltage_mon_1.t/ms, voltage_mon_1.v[0], c='b')
for t in spike_mon_1.t:
    axvline(t/ms, c='b', lw=2)

plot(voltage_mon_2.t/ms, voltage_mon_2.v[0], c='y', ls='--')
for t in spike_mon_2.t:
    axvline(t/ms, c='y', lw=2, ls='--')

ylim(-0.1, 0.04)
xlabel('Time (ms)')
ylabel('v')
show()

As you can see, neuron 2 is now affected by neuron 1, but it isn't ever excited enough to fire. How can we make neuron 2 excited enough to fire without changing any of its parameters? Copy the block above and test your theory.